# COM3029 Group Project

## Project Description

This project will aim to deliver a chatbot that will act as a conversational diary. The user will be able to add entries to their own digital diary through natural conversation with the chatbot. Additionally, the chatbot will allow the user to request details regarding previous entries in the diary. Each user will be able to be personally identifiable to the chatbot by providing the chatbot with their name and a special phrase or word that will unlock their diary information.

The diary will store user information regarding places they visited, people they met and how they felt that day.

## Model Serving Decisions

For this project we wanted to deliver the project in such a way that it follows a realistic deployment process that would appropriate for the delivery of a production level application. 

Various model serving options will be explored to determine the best way to deliver the application.

###

In [4]:
from enum import Enum, auto

class STATE(Enum):
    GREETING = auto()
    RUNNING=auto()
    ASKED_NAME=auto()
    QUIT=auto()

In [6]:
import requests
import json

In [ ]:
data = {}
state = 0
def get_response(url,data):
    response = requests.post(url,json=data)
    print(response.text)
    response = json.loads(response.text)
    return response["response"], response["state"]

def format_bot_response(response):
    print("Bot: {}".format(response))

data={"msg":None}
response, state= get_response('http://localhost:5000/start_greeting',data)
format_bot_response(response)


while state !=-1:
    data["msg"]=input("User: ")
    response, state= get_response('http://localhost:5000/get_response',data)
    format_bot_response(response)

print("Client closed")